In [261]:
import requests as rq
import numpy as np
from scipy.optimize import fsolve


In [262]:
# CONSTANTS
YEARS_TIL_GRAD = 6  # years
MASTER_YEARS = 2
RAISE_RATE = 1.03  # percent normalized
INTEREST_RATE = 1.03  # percent normalized

MONTHLY_SALARY = 20_000 * 2  # THB
BASE_INCOME = MONTHLY_SALARY * 12  # THB, not full income but sparable income
STARTING_BALANCE = 5_000_000  # THB

TARGET_FROM_CURRENCIES = ["USD", "EUR", "SGD", "GBP", "CAD", "AUD"]
TARGET_TO_CURRENCY = "THB"


# Computational Stuff


In [263]:
# 1 <key currency> converts to
current_exchange_rate = {
    "USD": 35.6,
    "EUR": 36.33,
    "SGD": 25.82,
    "GBP": 45.06,
    "CAD": 27.69,
    "AUD": 24.89
}

# API keys, ignore if unneeded
YOUR_USER = ""
YOUR_KEY = ""


def query_currency_conversion() -> dict[str, float]:
    if len(current_exchange_rate) == len(TARGET_FROM_CURRENCIES):
        return current_exchange_rate

    currencies_map = {}
    for currency in TARGET_FROM_CURRENCIES:
        params = {
            # If this gets stolen, it's fine. I didn't bind the account with my credit card.
            "user-id": YOUR_USER,
            "api_key": YOUR_KEY,
            "from-type": currency,
            "from-value": 1,
            "to-type": TARGET_TO_CURRENCY
        }
        # limited use, just use once then comment out
        response = rq.post("https://neutrinoapi.net/convert", params)
        conversion_rate = float(response.json()["result"])
        currencies_map[currency] = conversion_rate
    return currencies_map


current_exchange_rate = query_currency_conversion()
for k, v in current_exchange_rate.items():
    print(f"1 {k} converts to {v} THB")


1 USD converts to 35.6 THB
1 EUR converts to 36.33 THB
1 SGD converts to 25.82 THB
1 GBP converts to 45.06 THB
1 CAD converts to 27.69 THB
1 AUD converts to 24.89 THB


In [264]:
class ActualCost:
    """
    Actual yearly and total cost of staying in grad school for N years
    """

    def __init__(self, pyc: float, og_currency: str):
        self.per_year_cost = pyc * current_exchange_rate[og_currency]
        self.total_cost = pyc * YEARS_TIL_GRAD * \
            current_exchange_rate[og_currency]

    def __repr__(self):
        N = f"Number of years expected to do program: {YEARS_TIL_GRAD} years"
        ttm = f"Total cost of grad school: {'{:,.2f}'.format(self.total_cost)} THB"
        pym = f"Yearly cost of grad school: {'{:,.2f}'.format(self.per_year_cost)} THB"
        return f"=== Actual Cost ===\n{N}\n{ttm}\n{pym}\n"


In [265]:
class SteadyStartResult:
    """
    Imagine you have N bank accounts, N representing how long you stay in grad school.
    Account 1 pays for year 1, account 2 pays for year 2, ..., account N pays for year N.

    Gives how much you'd need to put in account i for i in [1..=N] then sum.
    IRL, this probably will be a little bit more than what you actually need, ignoring inflation
    """

    def __init__(self, tsm: float, ysm: list[float]):
        self.total_starting_money = np.ceil(tsm * 100) / 100
        self.yearly_starting_money = list(np.ceil(np.array(ysm) * 100) / 100)

    def __repr__(self):
        tsm = f"Total money needed: {'{:,}'.format(self.total_starting_money)} THB"
        ysm = '\t\n'.join(['Year {:}: {:,} THB'.format(i+1, cy)
                          for i, cy in enumerate(self.yearly_starting_money)])
        return f"=== Steady Start Result ===\n{tsm}\n{ysm}\n"


def compute_steady_start(per_year_cost: float, og_currency: str) -> SteadyStartResult:
    # converting currency
    per_year_cost = per_year_cost * current_exchange_rate[og_currency]

    # for the N bank accounts
    targets = np.array([per_year_cost] * YEARS_TIL_GRAD)
    multipliers = np.cumprod([INTEREST_RATE] * YEARS_TIL_GRAD)

    # computing how much needed for each account
    yearly_target = targets / multipliers
    total_target = np.sum(yearly_target)

    return SteadyStartResult(total_target, list(yearly_target))


In [266]:
class EnoughStartResult:
    """
    In each year, roughly speaking, you have remaining money from last yea, earn this year's income
    (or spare part of this year's income), pay tuition, and get interest.

    We can compute how much money would be good at the start so that with steady income,
    constant tuition, and constant interest rate, you will be able to perfectly pay for grad school.
    """

    def __init__(self, gs: float):
        self.good_start = gs

    def __repr__(self):
        ybi = f"Yearly base income: {'{:,.2f}'.format(BASE_INCOME)} ({'{:,.2f}'.format(BASE_INCOME / 12)} monthly)"

        formatted_str = np.format_float_positional(self.good_start)
        formatted_float = "{:,.2f}".format(float(formatted_str))
        smn = f"Starting money needed: {formatted_float} THB"

        return f"=== Enough Start Result ===\n{ybi}\n{smn}\n"


def compute_enough_start(per_year_cost: float, og_currency: str) -> EnoughStartResult:
    # converting currency
    tuition = per_year_cost * current_exchange_rate[og_currency]

    def income_of_year(n: int) -> float:
        return BASE_INCOME * RAISE_RATE ** (n - 1)

    # to be used for optimization
    def find_opt_starter(v: list[float]):
        starting_balance = v[0]
        yr_table: dict[int, float] = {}  # year to balance

        # how much money would your bank account have left at the end of year n
        def remaining_of_year(n: int) -> float:
            if n in yr_table:
                return yr_table[n]
            elif n == 1:
                yr_table[n] = (starting_balance +
                               income_of_year(n) - tuition) * INTEREST_RATE
                return yr_table[n]
            else:
                yr_table[n] = (remaining_of_year(n - 1) +
                               income_of_year(n) - tuition) * INTEREST_RATE
                return yr_table[n]

        # to be able to pay the tuition for N years, balance of the last year should be 0
        return remaining_of_year(YEARS_TIL_GRAD)

    # solve eqn
    result = fsolve(find_opt_starter, [
                    (tuition - BASE_INCOME) * YEARS_TIL_GRAD])
    return EnoughStartResult(result[0])


In [267]:
class GoodIncomeResult:
    """
    Suppose you have B THB in your bank account, you want to find out how much base (starting)
    income you need in order to pay for your tuition, with raises accounted
    """

    def __init__(self, gi: float):
        self.good_income = gi

    def __repr__(self):
        start = f"Starting balance: {'{:,.2f}'.format(STARTING_BALANCE)} THB"
        formatted_str = np.format_float_positional(self.good_income)
        formatted_float = "{:,.2f}".format(float(formatted_str))
        smn = f"Base income needed: {formatted_float} THB ({'{:,.2f}'.format(float(formatted_str) / 12)} THB monthly in first year)"
        return f"=== Good Income Result ===\n{start}\n{smn}\n"


def compute_good_income(per_year_cost: float, og_currency: str) -> GoodIncomeResult:
    # converting currency
    tuition = per_year_cost * current_exchange_rate[og_currency]

    # to be used for optimization
    def find_opt_income(v: list[float]):
        # the actual thing we wanna optimize for
        def income_of_year(n: int) -> float:
            return v[0] * RAISE_RATE ** (n - 1)

        # same thing as above, want last year balance to be 0 given starting balance
        yr_table: dict[int, float] = {}  # year to balance

        def remaining_of_year(n: int) -> float:
            if n in yr_table:
                return yr_table[n]
            elif n == 1:
                yr_table[n] = (STARTING_BALANCE +
                               income_of_year(n) - tuition) * INTEREST_RATE
                return yr_table[n]
            else:
                yr_table[n] = (remaining_of_year(n - 1) +
                               income_of_year(n) - tuition) * INTEREST_RATE
                return yr_table[n]

        # remaining of last year
        return remaining_of_year(YEARS_TIL_GRAD)

    # solve eqn
    result = fsolve(find_opt_income, [
                    (tuition - BASE_INCOME) * YEARS_TIL_GRAD])
    return GoodIncomeResult(result[0])


In [268]:
def cost_of_university(per_year_cost: float, og_currency: str):
    print(ActualCost(per_year_cost, og_currency))
    print(compute_steady_start(per_year_cost, og_currency))
    print(compute_enough_start(per_year_cost, og_currency))
    print(compute_good_income(per_year_cost, og_currency))


# Carnegie Mellon University @ Pittsburgh, USA

- CS Ph.D. gets full financial support while in good academic standing
- Seems to be offering a lot of financial aid overall


## Computer Science

- Admissions Link: https://www.cs.cmu.edu/academics/graduate-admissions
- Requirements:
  - Online Application
  - GRE (optional but recommended)
  - IELTS
  - Transcript
  - Resume/CV
  - Statement of Purpose
  - 3 Recommendation Letters
- Doesn't require master's to apply to Ph.D.
- Ph.D. takes about 5-6 years

### Algorithms, Combinatorics, and Optimization (ACO)

- Link: https://www.cmu.edu/math/aco/index.html
- By Tepper School of Business (Operations Research Group), Computer Science Department (Algorithms and Complexity group),
  and Department of Mathematical Sciences (Discrete Mathematics group)
- Study of math-structure of discrete objects, design and analysis of algorithms in graph theory, combinatorial optimization,
  integer programming, polyhedral theory, computational algebra, geometry, and number theory
  - Does papers on graph+matroid theories, linear programming, network flows, matchings in graphs
- Apply through any of the three participating departments but selected by ACO faculty
- Financial support under same conditions as other Ph.D. programs

### Machine Learning (Neural Computation)

- Link: https://www.ml.cmu.edu/about/
- Need to enter as Ph.D. for ML first, then choose to join with NC
- Joint with Program in Neural Computation (PNC) from Neuroscience Institute

## Physics

- Admissions Link: https://www.cmu.edu/physics/graduate-program/admission.html
- M.S. in Physics DNE here :P
- Requirements
  - Online Application
  - Transcript
  - 3 Letters of Recommendations
  - IELTS
  - GRE General or GRE Subject (optional but recommended)
- Preperation (Textbooks)
  - Intermediate Mechanics (Thornton & Marion)
  - Thermodynamics and Statistical Physics (Callen, Guenalt)
  - Atomic Physics (Eisberg & Resnick)
  - Elementary Quantum Mechanics (Griffiths)
  - Modern Physics Laboratory
  - Electronics
  - Advanced Calculus

### Computational Physics

- Link: https://www.cmu.edu/physics/research/comp-phys.html
- Computer simulation and data mining/analysis
- Cooperates with Pittsburgh Supercomputing Center

### Astrophysics & Cosmology

- Link: https://www.cmu.edu/physics/research/astro-cosmo.html
- Theoretical, computation, and observational cosmology
- Has a approx. 1500 core computer cluster available for use

### Nuclear & Particle Physics

- Link: https://www.cmu.edu/physics/research/nuclear-particle.html
- Understanding the building blocks of the universe
- Elementary particles and interactions between them
- Improve understanding of several mysteries within the otherwise very successful std model of particle physics
- Internal structure of protons and neutrons (quarks and gluonic field and their confinement)


In [269]:
college_of_science = 76122
school_of_computer_science = 28872 + 46400
tepper_school_of_business = 47000 + 450 + 15967

cmu_average = np.average(
    [college_of_science, school_of_computer_science, tepper_school_of_business])

cost_of_university(cmu_average, "USD")


=== Actual Cost ===
Number of years expected to do program: 6 years
Total cost of grad school: 15,294,543.20 THB
Yearly cost of grad school: 2,549,090.53 THB

=== Steady Start Result ===
Total money needed: 13,808,911.43 THB
Year 1: 2,474,845.18 THB	
Year 2: 2,402,762.31 THB	
Year 3: 2,332,778.95 THB	
Year 4: 2,264,833.93 THB	
Year 5: 2,198,867.89 THB	
Year 6: 2,134,823.2 THB

=== Enough Start Result ===
Yearly base income: 480,000.00 (40,000.00 monthly)
Starting money needed: 11,343,178.77 THB

=== Good Income Result ===
Starting balance: 5,000,000.00 THB
Base income needed: 1,537,196.46 THB (128,099.71 THB monthly in first year)



# Massachusetts Institute of Technology @ Massachusetts, USA

- Has fundings from each department to support doctoral students for the cost of full tuition, stipend, and health insurance
  - Funding may take form of research, instructor, and teaching assistantships, fellowships, etc.
  - Considered for awards once accepted into graduated program
  - Master's level is more limited


## Electrical Engineering and Computer Science

- Admissions: https://oge.mit.edu/programs/electrical-engineering-and-computer-science/
- No need for M.S. to apply, but will need to work on it first
- Requires at least a Bachelor's Degree

### CS: Quantum Computing, Communication, and Sensing

- Everything from new materials to superconducting devices and quantum computers to theory
- Link: https://www.eecs.mit.edu/research/explore-all-research-areas/quantum-computing-communication-and-sensing/

### AID: Optimization and Game Theory

- Developing efficient and scalable algorithms for solving large scale optimization problems.
- Link: https://www.eecs.mit.edu/research/explore-all-research-areas/optimization-and-game-theory/

### CS: Programming Languages and Software Engineering

- New approaches to programming: new languages, tools, methodologies
- Link: https://www.eecs.mit.edu/research/explore-all-research-areas/programming-languages-and-software-engineering/

### AID: Artificial Intelligence and Machine Learning

- Advancing how machines learn, predict, control, while making them secure, robust, and trustworthy
  - ML theory, statistical learning, DL, reinforcement learning, hardware of ML, etc.
- Link: https://www.eecs.mit.edu/research/explore-all-research-areas/artificial-intelligence-and-machine-learning/

## Physics

- Admissions: https://oge.mit.edu/programs/physics/
- Seems like no need for M.Sc. to apply
- Requirements
  - Online Application ($75)
  - Transcript (unofficial is sufficient)
  - GRE Tests, subject or general (optional)
  - 3 Letters of Recommendation
  - Statement of Objectives
- Good-to-knows
  - Past research experience, publications, awards, honors are useful
  - Successful applicants usually have B.Sc. Physics, or have taken many physics classes
  - Ph.D. takes 3-7 years, averaging 5.6 years
  - In 2021, 1500 applicants => admitted just over 100 => 2021 incoming class has 65 students

### Astrophysics Observation, Instrumentation, and Experiment

- Link: https://physics.mit.edu/research-areas/astrophysics-observation-instrumentation-and-experiment/
- Focuses on
  - Structure formation and epoch of reionization
  - Extragalactic and galactic astrophysics
  - The first stars
  - Neutron stars and black holes
  - Exoplanets
- Faculties developing new instruments & techniques pushing the frontiers of such studies

### Nuclear and Particle Physics Experiment

- Link (Nuclear): https://physics.mit.edu/research-areas/nuclear-physics-experiment/
- Link (Particle): https://physics.mit.edu/research-areas/particle-physics-experiment/
- Research questions:
  - New forces or particles beyond already known?
  - How do fundamental forces interact?
  - Dark matter?
  - Matter and anti-matter?
  -

### Quantum Information Science

- Link: https://physics.mit.edu/research-areas/quantum-information-science/
- Research:
  - Quantum Algorithms and Complexity
  - Quantum Information Theory
  - Measurement and Control
  - Applications and Connections

## Computational Science and Engineering

- Admissions: https://cse.mit.edu/admissions/phd/
- Requirements:
  - 3 letters of recommendation
  - Transcript
  - Complete subjects taken
  - Statement of objectives
  - GRE General (optional)
  - IELTS
  - Resume/CV
- Doesn't require Master's unless admitting to Aeronautics and Astronauts or Mechanical Engineering

### Computational Modeling and Simulation

- Link: https://cse.mit.edu/research_categories/computational-modeling-and-simulation/

### Numerical Algorithms and Scientific Computing

- Developing fast, scalable algorithms for computational problems
- Theoretical analysis of complexity and convergence
- Link: https://cse.mit.edu/research_categories/numerical-algorithms-and-scientific-computing/


In [270]:
mit_average = 126450  # (12-month cost)
cost_of_university(mit_average, "USD")


=== Actual Cost ===
Number of years expected to do program: 6 years
Total cost of grad school: 27,009,720.00 THB
Yearly cost of grad school: 4,501,620.00 THB

=== Steady Start Result ===
Total money needed: 24,386,137.35 THB
Year 1: 4,370,504.86 THB	
Year 2: 4,243,208.6 THB	
Year 3: 4,119,620.0 THB	
Year 4: 3,999,631.07 THB	
Year 5: 3,883,136.96 THB	
Year 6: 3,770,035.88 THB

=== Enough Start Result ===
Yearly base income: 480,000.00 (40,000.00 monthly)
Starting money needed: 22,237,721.47 THB

=== Good Income Result ===
Starting balance: 5,000,000.00 THB
Base income needed: 3,352,953.58 THB (279,412.80 THB monthly in first year)



# California Institute of Technology @ California, USA

- Most departments don't have M.S. available (perfect)
- Requirements
  - Bachelor's degree to apply for graduate
  - GRE (optional)
  - Statement of purpose
  - 3 Letters of recomendation
  - Transcripts
  - Resume or CV, include (if available):
    - Teaching, professional, vocational experience
    - Research experience
    - Major academic recognitions, honors
    - Professional activities
    - Significant extracurricular activities
    - Conference presentations
    - Publications
  - IELTS
- Most graduate gets full funding for graduate education
  - All doctoral students have full financial support given satisfactory academic progress
  - No need for separate application for requesting financial aid
- 98% of grad students get a package of merit-based financial support for full tuition and provides stipend


## Computing and Mathematical Sciences

- Ph.D. oriented
- Minimum of 3 academic years

### Quantum Information and Computation

- Link: https://www.cms.caltech.edu/research/quantum-information-and-computation
- Quantum Science and Engineering Link: https://qse.caltech.edu/
- Hosts of Institute for Quantum Information and Matter (IQIM)
  - Joint with Physics as well
- Works on theoretical aspects of quantum computing:
  - Complexity theory
  - Cryptography
  - Algorithms
  - Benchmarking
  - Error correction

### Scientific Computing and Numerical Analysis

- Link: https://www.cms.caltech.edu/research/scientific-computing-and-numerical-analysis
- Research:
  - Contributes in ODE and PDE theory, numerical algorithms and analysis, HPC and ML
  - Specific applications for fluid-dynamics, solid mechanics, electromagnetism, spectral methods, multiple scale problems
  - Large-scale matrix computations and optimization problems
  - More!

### Optimization

- Link: https://www.cms.caltech.edu/research/optimization
- Many challenges in computation mathematics can be expressed as constrained optimization
- Also primary language for ML and automatic control
- Research:
  - Sparce optimization, linalg computations, semidefinite programming
  - More!

### Theoretical Computer Science

- Link: https://www.cms.caltech.edu/research/theoretical-computer-science
- Traditional algorithms and complexity + pure math (algebra, combinatorics, geometry, probability)
- Research:
  - Coding and communication, combinatorics and probability, theoretical ML, algorithmic game theory
  - Algorithms and complexity with connections to algebra + algorithms for matrix multiplication with group theory and representation theory
  - Quantum complexity and cryptography, especially in quantum interactive proofs
  - Classical verification of quantum computation + intersection of quantum computation and cryptography

### Artificial Intelligence and Machine Learning

- Link: https://www.cms.caltech.edu/research/artificial-intelligence-and-machine-learning
- Research:
  - Generalizable and robust deep learning methods for PDE and other scientific simultations
  - Experimental methods for measuring bias in AI
  - Computer vision

## Physics

- Link for all: https://pma.caltech.edu/research-and-academics/physics/physics-research
- **Nuclear Physics**
- **Observational Astrophysics**


In [271]:
on_campus_single = 93226
on_campus_shared2 = 86686
on_campus_shared4 = 85186
off_campus_shared2 = 93166

caltech_average = np.average(
    [on_campus_single, on_campus_shared2, on_campus_shared4, off_campus_shared2])

cost_of_university(caltech_average, "USD")


=== Actual Cost ===
Number of years expected to do program: 6 years
Total cost of grad school: 19,131,297.60 THB
Yearly cost of grad school: 3,188,549.60 THB

=== Steady Start Result ===
Total money needed: 17,272,983.62 THB
Year 1: 3,095,679.23 THB	
Year 2: 3,005,513.81 THB	
Year 3: 2,917,974.58 THB	
Year 4: 2,832,985.03 THB	
Year 5: 2,750,470.9 THB	
Year 6: 2,670,360.1 THB

=== Enough Start Result ===
Yearly base income: 480,000.00 (40,000.00 monthly)
Starting money needed: 14,911,173.12 THB

=== Good Income Result ===
Starting balance: 5,000,000.00 THB
Base income needed: 2,131,862.19 THB (177,655.18 THB monthly in first year)



# University of Waterloo @ Ontario, Canada

- Can get financial support but seems to not be full tuition + stipend like above universities
- Can find and get scholarship, done automatically once admitted


## Computer Science (or with Quantum Information)

- Requirements are similar, but application seems separate
- Admits in Fall, Winter, and Spring
- Requirements:
  - Minimum requirements:
    - Master's degree in CS with 78% average
    - Undergraduate may apply directly into Ph.D. but have to _basically be a god_
    - Ph.D. may be admitted into Master of Mathematics program first, then transfer to Ph.D. before completing that if performance warrants
  - Resume
  - Supplementary information form
  - Transcripts
  - 3 letters of recommendation
  - IELTS
  - GRE (optional)

### Quantum Information

- Link: https://uwaterloo.ca/institute-for-quantum-computing/programs
- Research:
  - Quantum algorithms
  - Quantum complexity theory
  - Quantum zero-knowledge
  - Theory of quantum information
  - Quantum security models

### Quantum Computing

- Link: https://cs.uwaterloo.ca/research/research-areas/quantum-computing
- Models of computation and communication that harness the power of quantum mechanics
- Focus
  - Theoretical aspects of quantum computing
  - Design and analysis of quantum algorithms
  - Cryptographyic protocols
  - Quantum fault tolerance
  - Various issues in information and complexity theory

### Algorithms and Complexity

- Link: https://cs.uwaterloo.ca/research/research-areas/algorithms-and-complexity
- Study of design, analysis, and implementation of algorithms
- Research:
  - Algorithm types and areas of applications
  - Limits of efficient computation measured in terms of time, space, and other resrouces such as randomness
  - Complexity theory like Kolmogorov complexity and cryptography

### Scientific Computing

- Link: https://cs.uwaterloo.ca/research/research-areas/scientific-computation

* Research:
  - Methods and software for supporting applications in scientific areas
  - Develop and analyze algorithms for solution of large sparce matrices, large-scale optimization, least-squares problems, numerical solution to PDE
  - Coming up with effective methods embedded in efficient software


In [272]:
living_costs = 6940 * 3  # per term, 3 terms
cs_phd = 7396 * 3
cs_master = 7696 * 3

waterloo_avg = np.average([cs_master + living_costs] * MASTER_YEARS +
                          [cs_phd + living_costs] * (YEARS_TIL_GRAD - MASTER_YEARS))

cost_of_university(waterloo_avg, "CAD")


=== Actual Cost ===
Number of years expected to do program: 6 years
Total cost of grad school: 7,195,191.12 THB
Yearly cost of grad school: 1,199,198.52 THB

=== Steady Start Result ===
Total money needed: 6,496,287.97 THB
Year 1: 1,164,270.41 THB	
Year 2: 1,130,359.62 THB	
Year 3: 1,097,436.53 THB	
Year 4: 1,065,472.36 THB	
Year 5: 1,034,439.18 THB	
Year 6: 1,004,309.89 THB

=== Enough Start Result ===
Yearly base income: 480,000.00 (40,000.00 monthly)
Starting money needed: 3,811,176.60 THB

=== Good Income Result ===
Starting balance: 5,000,000.00 THB
Base income needed: 281,862.77 THB (23,488.56 THB monthly in first year)



# University of Wisconsin, Madison @ Wisconsin, USA

- Application into graduate programs are the same thing, only bachelor's degree is required
- Requirements:
  - 3 Letters of recommendation
  - GRE General scores
  - Transcript
  - IELTS
  - Resume/CV
- Graduate students are eligible for many ways of financial support
  - TA
  - RA
  - Fellowship
- All admitted Ph.D. students are provided with a guarantee of 5 years of financial support
- Mostly awarded at time of admission


## Computer Science

# University of California, Berkeley @ California, USA


# University of Chicago @ Chicago, USA


# Delft University of Technology @ Delft, Netherlands


# Stanford University @ California, USA


# University of Colorado Bouder @ Colorado, USA


# University of Bristol @ Bristol, England


# University of Cambridge @ Cambridge, England


# University of Sydney @ Sydney, Australia


# University of Tokyo @ Tokyo, Japan


# National University of Singapore @ Singapore
